In [40]:
import pandas as pd
from graphdatascience import GraphDataScience

In [41]:
# neo4j desktop m1, 5.3.0
host = "bolt://localhost:7687"
user = "neo4j"
password= "j4oenj4oen"

gds = GraphDataScience(host, auth=(user, password))
print(gds.version())

2.3.2


In [43]:
# # (opt1) project graph in memory by pandas read_csv
# cora_papers = "data/cora_papers.csv"
# cora_cites = "data/cora_cites.csv"
# papers = pd.read_csv(cora_papers, header=None)
# cites = pd.read_csv(cora_cites, header=None)

In [44]:
# # create nodes dataframe from csv
# subject_to_id = {
#     "Neural_Networks": 0,
#     "Rule_Learning": 1,
#     "Reinforcement_Learning": 2,
#     "Probabilistic_Methods": 3,
#     "Theory": 4,
#     "Genetic_Algorithms": 5,
#     "Case_Based": 6,
# }
# nodes = pd.DataFrame().assign(
#     nodeId=papers[0],
#     labels="paper",
#     subject=papers[1].replace(subject_to_id),
#     features=papers.iloc[:, 2:].apply(list, axis=1),
# )
# nodes.head()

,nodeId,labels,subject,features
0,31336,paper,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1061127,paper,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,1106406,paper,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,13195,paper,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,37879,paper,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [45]:
# # create edges dataframe from csv
# dir_relationships = pd.DataFrame().assign(sourceNodeId=cites[0], targetNodeId=cites[1], relationshipType="CITES")
# inv_relationships = pd.DataFrame().assign(sourceNodeId=cites[1], targetNodeId=cites[0], relationshipType="CITES")
# relationships = pd.concat([dir_relationships, inv_relationships]).drop_duplicates()
# relationships.head()

,sourceNodeId,targetNodeId,relationshipType
0,35,1033,CITES
1,35,103482,CITES
2,35,103515,CITES
3,35,1050679,CITES
4,35,1103960,CITES


In [46]:
# # create in-memory graph
# G = gds.alpha.graph.construct("cora-graph", nodes, relationships)
# gds.graph.list()

,degreeDistribution,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema,schemaWithOrientation
0,"{'p99': 19, 'min': 1, 'max': 168, 'mean': 3.89...",cora-graph,neo4j,34 MiB,35685679,2708,10556,{'jobId': '4fac5b18-fe24-4c34-8a62-f610dc605eb...,0.00144,2023-06-28T23:02:10.258404000+09:00,2023-06-28T23:02:10.291832000+09:00,"{'graphProperties': {}, 'relationships': {'CIT...","{'graphProperties': {}, 'relationships': {'CIT..."


In [ ]:
# (opt2) write to neo4j by LOAD CSV cypher
# file should be placed in import/ if using e.g., LOAD CSV FROM $url AS row FIELDTERMINATOR ' ' in cypher ('url': 'file:///...')


In [47]:
# free up memory
G.drop()

query = """
MATCH (n) DETACH DELETE n
"""
gds.run_cypher(query)
gds.close()